# Problem Statement

When a field gets added to a live REDCap, CRPs would have to manually go through all the pre-existing records to fill out that field. 

Similarly, when a field currently has wrong value for multiple records and need its value changed, CRPs need to manually go in and fix these data points.

This manual process may introduce human errors and take too much time for the CRPs.

# Solution

This notebook lets the user specify the filter on the records and the fields that need new/updated values. 

Then, it exports only the records that need to be changed. Changes are made on this export before getting re-imported to the project.

**The notebook makes these changes in the Test project before actually making the changes in the Target project. Pay attention to the changes in the Test project and make sure of data integrity before moving forward with the Target project.**

# Instructions

### **TEST WITH TEST PROJECT BEFORE MAKING CHANGES TO THE TARGET PROJECT.**
### Instructions: 
1. Export entire target project as REDCap XML file (containing metadata & data).
   - `Data Exports, Reports and, Stats` > `Other Export Options` > `Export Entire Project...`
2. Import above export into test project.
   - `Data Import` > `CDISC ODM (XML) import` 
   - There's a chance the file might be too big for importing. In this case, use the notebook `copy_records_redcap.ipynb`.
3. Run this notebook.

# Code
### 1. Setup: Specify condition & fields, get project and project data

In [1]:
import AMBRA_Backups

In [2]:
# TEST PROJECT
test_project_name = 'Personal Test'
test_project = AMBRA_Backups.redcap_funcs.get_redcap_project(test_project_name)

# TARGET PROJECT
target_project_name = 'I3C DECADE - Image Tracking'
target_project = AMBRA_Backups.redcap_funcs.get_redcap_project(target_project_name)

In [3]:
# FILTERS
filter_logic = '''
   [patient_status_1] = ''
'''

# FIELDS DICTIONARY WITH NEW VALUES NEEDED
# Format:
# {
#     'field_name': [field_name],
#     'old_value':  [old_value],
#     'new_value':  [new_value] 
# }
update_fields_dict = [
   {  'field_name': 'patient_status_1',
      'old_value': '',
      'new_value': '1'
   }
]

### 2. Testing with Test project

In [4]:
def update_fields(records: list, update_fields_dict: dict):
   '''
   Change the fields in `update_fields` into specified values in `records`.
   '''

   for record in records:
      for field in update_fields_dict:
         field_name = field['field_name']
         current_value = record[field_name]

         # Update value
         if current_value == field['old_value']:
            record[field_name] = field['new_value']


   return records

In [5]:
# Write current records of test project to Excel file for later comparison
test_records_df = test_project.export_records(format_type='df')
test_records_df.to_excel(f'{test_project_name}_before_update.xlsx')


In [6]:
# Get records with filters from test project
test_filtered_records = test_project.export_records(filter_logic=filter_logic, export_blank_for_gray_form_status=False)
test_filtered_records

[{'record_id': 'B00000610',
  'id': 'B00000610',
  'patient_status_1': '',
  'site': '2',
  'name': '',
  'dob': '',
  'mrn': '',
  'icf_upload': '',
  'scan_date': '2023-07-28 12:00',
  'in_epic': '',
  'created': '',
  'epic_order_status': '',
  'scheduling_email': '',
  'metallic_implants': '',
  'specify': '',
  'technologist_confirmation': '',
  'epic_upload': '',
  'outlook': '',
  'scheduling_complete': '2',
  'fmri_completed': '1',
  'patient_status': '',
  'sequences_acquired___1': '0',
  'sequences_acquired___2': '0',
  'sequences_acquired___3': '0',
  'e_prime': '1',
  'e_prime_collected': '',
  'physio': '1',
  'physio_collected': '',
  'qa_qc': '1',
  'routing': '1',
  'report_attached': '1',
  'share_drive': '1',
  'site_query': '0',
  'site_query_status': '',
  'email_cen': '1',
  'read': '1',
  'query': '3',
  'state_query': '',
  'data_check': '1',
  'query_generated': '',
  'query_generated_notes': '',
  'ambra_flow_complete': '2',
  'incidental_findings_reported': '0

In [7]:
# Update the records
test_updated_records = update_fields(test_filtered_records, update_fields_dict)
test_updated_records

[{'record_id': 'B00000610',
  'id': 'B00000610',
  'patient_status_1': '1',
  'site': '2',
  'name': '',
  'dob': '',
  'mrn': '',
  'icf_upload': '',
  'scan_date': '2023-07-28 12:00',
  'in_epic': '',
  'created': '',
  'epic_order_status': '',
  'scheduling_email': '',
  'metallic_implants': '',
  'specify': '',
  'technologist_confirmation': '',
  'epic_upload': '',
  'outlook': '',
  'scheduling_complete': '2',
  'fmri_completed': '1',
  'patient_status': '',
  'sequences_acquired___1': '0',
  'sequences_acquired___2': '0',
  'sequences_acquired___3': '0',
  'e_prime': '1',
  'e_prime_collected': '',
  'physio': '1',
  'physio_collected': '',
  'qa_qc': '1',
  'routing': '1',
  'report_attached': '1',
  'share_drive': '1',
  'site_query': '0',
  'site_query_status': '',
  'email_cen': '1',
  'read': '1',
  'query': '3',
  'state_query': '',
  'data_check': '1',
  'query_generated': '',
  'query_generated_notes': '',
  'ambra_flow_complete': '2',
  'incidental_findings_reported': '

In [8]:
# Reimport back to Test project
test_project.import_records(test_updated_records)

{'count': 260}